In [10]:
import utils as util
import numpy as np
from tensorflow.keras.datasets import mnist
from tqdm import tqdm

In [11]:
class Layers():
    class CONV():
        def __init__(self, inputs_channel, num_filters, kernel_size):
          self.F = num_filters
          self.K = kernel_size
          self.C = inputs_channel

          self.weights = 2*(np.random.rand(self.F,self.C, self.K, self.K)-0.5)
          self.bias = 2*(np.random.rand(self.F, 1)-0.5)

        def forward(self, inputs, padding, stride):
            C = inputs.shape[0]
            W = inputs.shape[1]+2*padding
            H = inputs.shape[2]+2*padding

            self.inputs = np.zeros((C, W, H))

            for c in range(inputs.shape[0]):
                self.inputs[c,:,:] = util.padder(inputs[c,:,:], padding)

            return util.conv2d(self.inputs, self.weights, self.bias, padding, self.K, self.F, stride)
          
        def backward(self, dy, stride, learning_rate):
            dy = util.back_tanh(dy)

            C, W, H = self.inputs.shape

            dx = np.zeros(self.inputs.shape)
            dw = np.zeros(self.weights.shape)
            db = np.zeros(self.bias.shape)

            if len(dy.shape)==2:
              dy = np.array([dy])
            F, W, H = dy.shape

            for f in range(F):
                for w in range(0, W-self.K, stride):
                    for h in range(0, H-self.K, stride):
                        dw[f,:,:,:]+=dy[f,w,h]*self.inputs[:,w:w+self.K,h:h+self.K]
                        dx[:,w:w+self.K,h:h+self.K]+=dy[f,w,h]*self.weights[f,:,:,:]

            for f in range(F):
                db[f] = np.sum(dy[f, :, :])

            self.weights -= learning_rate * dw
            self.bias -= learning_rate * db

            return dx
    
    class POOL():
        def __init__(self, pool_size):
            self.pool = pool_size
        
        def forward(self, data, stride):
            self.inputs = data
            return util.avg_pool(data,self.pool,stride)
        # change later
        def backward(self, dy):
            C, W, H = self.inputs.shape
            dx = np.zeros(self.inputs.shape)
            
            for c in range(C):
                for w in range(0, W, self.pool):
                    for h in range(0, H, self.pool):
                        st = np.argmax(self.inputs[c,w:w+self.pool,h:h+self.pool])
                        (idx, idy) = np.unravel_index(st, (self.pool, self.pool))
                        dx[c, w+idx, h+idy] = dy[c, w//self.pool, h//self.pool]
            return dx
    
    class DENSE():
        def __init__(self, num_inputs, num_outputs, act):
            self.weights = 2*(np.random.rand(num_inputs, num_outputs)-0.5)
            self.bias = 2*(np.random.rand(num_outputs, 1)-0.5)
            self.act=act
        
        def forward(self, data):
            self.inputs = data
            if self.act == 'tanh':
              return util.tanh(util.vanilla(data,self.weights,self.bias))
            elif self.act == 'softmax':
              self.out = util.softmax(util.vanilla(data,self.weights,self.bias)[:,0])
              return self.out

        def backward(self, dy, learning_rate):
            # print("blah",dy.shape)
            if self.act == 'tanh':
                dy = util.back_tanh(dy).T
                dw = dy.dot(self.inputs.T)
                db = np.sum(dy, axis=1, keepdims=True)
                dx = np.dot(dy.T, self.weights.T)
            elif self.act=='softmax':
                # print("bleh",(self.out.T - dy.reshape(dy.shape[0],1)).shape)
                # print(self.out.T)
                # print(dy.reshape(dy.shape[0],1))
                # print(self.out.T - dy.reshape(dy.shape[0],1))
                # dy = self.out.T - dy.reshape(dy.shape[0],1)
                # dy = np.square(self.out.T - dy).reshape(dy.shape[0],1)
                out_num = dy.shape[0]
                p = dy.reshape(1,out_num)*self.out.T
                dy = -np.log(p).T
                dw = dy.dot(self.inputs.T)
                db = np.sum(dy, axis=1, keepdims=True)
                dx = np.dot(dy.T, self.weights.T)

            # print("blah",dy.shape)
            # if dy.shape[0] == self.inputs.shape[0]:
            #     dy = dy.T
            # print("blah",dy.shape)
            # dw = dy.dot(self.inputs)
            # db = np.sum(dy, axis=1, keepdims=True)
            # dx = np.dot(dy.T, self.weights.T)

            self.weights -= learning_rate * dw.T
            self.bias -= learning_rate * db

            return dx

In [12]:
d = {0: [1, 1, 1, 0, 0, 0],
 1: [0, 1, 1, 1, 0, 0],
 2: [0, 0, 1, 1, 1, 0],
 3: [0, 0, 0, 1, 1, 1],
 4: [1, 0, 0, 0, 1, 1],
 5: [1, 1, 0, 0, 0, 1],
 6: [1, 1, 1, 1, 0, 0],
 7: [0, 1, 1, 1, 1, 0],
 8: [0, 0, 1, 1, 1, 1],
 9: [1, 0, 0, 1, 1, 1],
 10: [1, 1, 0, 0, 1, 1],
 11: [1, 1, 1, 0, 0, 1],
 12: [1, 1, 0, 1, 1, 0],
 13: [0, 1, 1, 0, 1, 1],
 14: [1, 0, 1, 1, 0, 1],
 15: [1, 1, 1, 1, 1, 1]}

In [13]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
print('X_train: ' + str(train_X.shape))
print('Y_train: ' + str(train_y.shape))
print('X_test:  ' + str(test_X.shape))
print('Y_test:  ' + str(test_y.shape))

X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


In [14]:
layers = Layers()
C1 = layers.CONV(1,6,5)
S2 = layers.POOL(2)
C3 = layers.CONV(6,16,5)
S4 = layers.POOL(2)
C5 = layers.CONV(16,120,5)
F6 = layers.DENSE(120,84,'tanh')
F7 = layers.DENSE(84,10,'softmax')
def forward_pass(im):
  fmap1 = util.tanh(C1.forward(np.array([im]),2,1))
  # print("C1 Done", fmap1.shape)
  fmap2 = S2.forward(fmap1,2)
  # print("S2 Done", fmap2.shape)
  fmap3 = util.tanh(C3.forward(fmap2,0,1))
  # print("C3 Done", fmap3.shape)
  fmap4 = S4.forward(fmap3,2)
  # print("S4 Done", fmap4.shape)
  fmap5 = util.tanh(C5.forward(fmap4,0,1))
  # print("C5 Done", fmap5.shape)
  fmap6 = F6.forward(fmap5[:,0])
  # print("F6 Done", fmap6.shape)
  fmap7 = F7.forward(fmap6)
  return [fmap1, fmap2, fmap3, fmap4, fmap5, fmap6, fmap7]


all_layers = [C1, S2, C3, S4, C5, F6, F7]

In [15]:
output = forward_pass(train_X[0])

## Training

In [16]:
def test(ind, to_print=True):
    val = forward_pass(train_X[ind])
    if to_print:
        print("True Label:", train_y[ind], "Predicted:", np.argmax(val[-1]))
    return train_y[ind], np.argmax(val[-1])

In [17]:
def train(epochs):
    for epoch in range(epochs):
        print("Epoch:",epoch)
        for i, im in tqdm(enumerate(train_X[:10000])):
            # if i%100==0:
              # print(i, end=" ")
            label = util.one_hot(train_y[i])
            fmaps = forward_pass(im)
            t = F7.backward(fmaps[-1], learning_rate = 0.05)
            t = F6.backward(t, learning_rate = 0.05)
            t = C5.backward(t, stride = 1, learning_rate = 0.05)
            t = S4.backward(t)
            t = C3.backward(t, stride = 1, learning_rate = 0.05)
            t = S2.backward(t)
            t = C1.backward(t, stride = 1, learning_rate = 0.05)
        acc = 0
        for i in tqdm(range(10000,11000)):
            x,y = test(i,to_print=False)
            if x==y:
                acc += 1
        acc/=10
        print("Acc:",acc,"percent")



In [19]:
train(20)

1it [00:00,  5.79it/s]

Epoch: 0


615it [01:32,  5.94it/s]

In [ ]:
acc = 0
for i in tqdm(range(0,100)):
  x,y = test(i,to_print=False)
  if x==y:
    acc += 1
acc/=100
acc

In [ ]:
output[-1]

In [ ]:
forward_pass(train_X[0])[-1]